In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
import string
import re
%matplotlib inline
pd.set_option('display.max_colwidth', 100)
import wordcloud

In [9]:
df = pd.read_csv('../data/results/data_sentiment.csv')

In [10]:
df

,date,id,text,user_name,tweets_clean,sentiment,score,label
0,2020-09-29 06:34:23,1310830261450539009,RT @carnecrudaradio: Quiero felicitar al alcalde @AlmeidaPP_ por su exitosa labor destruyendo @B...,alex vega,RT Quiero felicitar al alcalde por su exitosa labor destruyendo un servicio que funcionaba y que vu,"[{'label': 'NEGATIVE', 'score': 0.9859856963157654}]",0.985986,NEGATIVE
1,2020-09-29 07:01:33,1310837099189473280,"Señores de @BiciMAD @MADRID las bicis están muy descuidadas (lo saben). A menor uso de bicis, má...",Neuroneater,Se ores de las bicis est n muy descuidadas lo saben A menor uso de bicis m s de metro y bus y,"[{'label': 'NEGATIVE', 'score': 0.9817882180213928}]",0.981788,NEGATIVE
2,2020-09-29 07:43:50,1310847740386201600,"@JMDLatina Espero de este distrito no solo que proteja el mejor legado ambiental de Madrid, sino...",Andrés Pina,Espero de este distrito no solo que proteja el mejor legado ambiental de Madrid sino que incorpo...,"[{'label': 'NEGATIVE', 'score': 0.9809810519218445}]",0.980981,NEGATIVE
3,2020-09-29 07:53:20,1310850131344920576,RT @_AguilarM: @PlataformaEMT @BiciMAD @bcarabante @AlmeidaPP_ @EMTmadrid O la fecha de la últim...,ElMaNDaLoRiaNo,RT AguilarM O la fecha de la ltima OPE para Avituallamiento y Limpieza HACE,"[{'label': 'NEGATIVE', 'score': 0.9831892251968384}]",0.983189,NEGATIVE
4,2020-09-29 08:05:56,1310853301810888704,La misma vergüenza de TODOS los días. Una estación de @BiciMAD totalmente abandonada por @MADRID...,Diego Azul,La misma verg enza de TODOS los d as Una estaci n de totalmente abandonada por igua,"[{'label': 'NEGATIVE', 'score': 0.9943681955337524}]",0.994368,NEGATIVE
...,...,...,...,...,...,...,...,...
6944,2020-10-19 22:24:16,1318317065267253249,RT @MADRID: .@BiciMAD 🚲 llega en octubre a Latina y Fuencarral-El Pardo y crece en Ciudad Lineal...,Gbj,RT llega en octubre a Latina y Fuencarral El Pardo y crece en Ciudad Lineal con la puesta en mar...,"[{'label': 'POSITIVE', 'score': 0.7803354859352112}]",0.780335,POSITIVE
6945,2020-10-19 22:25:32,1318317385502367746,RT @Rita_Maestre: La foto que resume lo que ha hecho Almeida con @BiciMAD: https://t.co/fXV7f0PYr7,JaviSeCa,RT Maestre La foto que resume lo que ha hecho Almeida con,"[{'label': 'NEGATIVE', 'score': 0.9442549347877502}]",0.944255,NEGATIVE
6946,2020-10-19 22:26:53,1318317727140974598,RT @Rita_Maestre: La foto que resume lo que ha hecho Almeida con @BiciMAD: https://t.co/fXV7f0PYr7,J. L.,RT Maestre La foto que resume lo que ha hecho Almeida con,"[{'label': 'NEGATIVE', 'score': 0.9442549347877502}]",0.944255,NEGATIVE
6947,2020-10-19 22:27:59,1318318002102849536,RT @Rita_Maestre: La foto que resume lo que ha hecho Almeida con @BiciMAD: https://t.co/fXV7f0PYr7,Corto Maltés,RT Maestre La foto que resume lo que ha hecho Almeida con,"[{'label': 'NEGATIVE', 'score': 0.9442549347877502}]",0.944255,NEGATIVE


In [33]:
df['ADR_label'] = np.random.randint(0, 2, df.shape[0])

** X.X. Pre-processing text data **  
Most of the text data are cleaned by following below steps.

- Remove punctuations
- Tokenization - Converting a sentence into list of words
- Remove stopwords
- Lammetization/stemming - Tranforming any form of a word to its root word

In [13]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [14]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

df['text'] = df['text'].apply(lambda x: remove_punct(x))
df.head(10)

,date,id,text,user_name,tweets_clean,sentiment,score,label,ADR_label
0,2020-09-29 06:34:23,1310830261450539009,RT carnecrudaradio Quiero felicitar al alcalde AlmeidaPP por su exitosa labor destruyendo BiciMA...,alex vega,RT Quiero felicitar al alcalde por su exitosa labor destruyendo un servicio que funcionaba y que vu,"[{'label': 'NEGATIVE', 'score': 0.9859856963157654}]",0.985986,NEGATIVE,0
1,2020-09-29 07:01:33,1310837099189473280,Señores de BiciMAD MADRID las bicis están muy descuidadas lo saben A menor uso de bicis más de m...,Neuroneater,Se ores de las bicis est n muy descuidadas lo saben A menor uso de bicis m s de metro y bus y,"[{'label': 'NEGATIVE', 'score': 0.9817882180213928}]",0.981788,NEGATIVE,0
2,2020-09-29 07:43:50,1310847740386201600,JMDLatina Espero de este distrito no solo que proteja el mejor legado ambiental de Madrid sino q...,Andrés Pina,Espero de este distrito no solo que proteja el mejor legado ambiental de Madrid sino que incorpo...,"[{'label': 'NEGATIVE', 'score': 0.9809810519218445}]",0.980981,NEGATIVE,0
3,2020-09-29 07:53:20,1310850131344920576,RT AguilarM PlataformaEMT BiciMAD bcarabante AlmeidaPP EMTmadrid O la fecha de la última OPE par...,ElMaNDaLoRiaNo,RT AguilarM O la fecha de la ltima OPE para Avituallamiento y Limpieza HACE,"[{'label': 'NEGATIVE', 'score': 0.9831892251968384}]",0.983189,NEGATIVE,1
4,2020-09-29 08:05:56,1310853301810888704,La misma vergüenza de TODOS los días Una estación de BiciMAD totalmente abandonada por MADRID Al...,Diego Azul,La misma verg enza de TODOS los d as Una estaci n de totalmente abandonada por igua,"[{'label': 'NEGATIVE', 'score': 0.9943681955337524}]",0.994368,NEGATIVE,1
5,2020-09-29 08:13:49,1310855286345928706,RT carnecrudaradio Quiero felicitar al alcalde AlmeidaPP por su exitosa labor destruyendo BiciMA...,Juan talavera,RT Quiero felicitar al alcalde por su exitosa labor destruyendo un servicio que funcionaba y que vu,"[{'label': 'NEGATIVE', 'score': 0.9859856963157654}]",0.985986,NEGATIVE,1
6,2020-09-29 08:13:52,1310855298777837568,RT carnecrudaradio Quiero felicitar al alcalde AlmeidaPP por su exitosa labor destruyendo BiciMA...,Susie Vicha,RT Quiero felicitar al alcalde por su exitosa labor destruyendo un servicio que funcionaba y que vu,"[{'label': 'NEGATIVE', 'score': 0.9859856963157654}]",0.985986,NEGATIVE,0
7,2020-09-29 08:50:57,1310864629661343744,RT diegorebollo La misma vergüenza de TODOS los días Una estación de BiciMAD totalmente abandona...,Carlos Martin 🔻,RT rebollo La misma verg enza de TODOS los d as Una estaci n de totalmente abandonada por igual que,"[{'label': 'NEGATIVE', 'score': 0.9925712943077087}]",0.992571,NEGATIVE,1
8,2020-09-29 09:09:00,1310869173782958080,PDiazAlejo RobertoRR josemanuelco ManuelaCarmena Sí tampoco es que durante cuatro años crearan u...,Diego Fernández,R R co S tampoco es que durante cuatro a os crearan una infraest,"[{'label': 'NEGATIVE', 'score': 0.9570339918136597}]",0.957034,NEGATIVE,0
9,2020-09-29 09:12:47,1310870126628134912,BiciMAD Hola no he podido desanclar ninguna bici el mensaje de la app es que no tengo la cuenta ...,María Domínguez Díaz,Hola no he podido desanclar ninguna bici el mensaje de la app es que no tengo la cuenta activada...,"[{'label': 'NEGATIVE', 'score': 0.9886822700500488}]",0.988682,NEGATIVE,1


In [16]:
def tokenization(text):
    text = re.split('\W+', text)
    return text

df['Tweet_tokenized'] = df['text'].apply(lambda x: tokenization(x.lower()))
df.head()

,date,id,text,user_name,tweets_clean,sentiment,score,label,ADR_label,Tweet_tokenized
0,2020-09-29 06:34:23,1310830261450539009,RT carnecrudaradio Quiero felicitar al alcalde AlmeidaPP por su exitosa labor destruyendo BiciMA...,alex vega,RT Quiero felicitar al alcalde por su exitosa labor destruyendo un servicio que funcionaba y que vu,"[{'label': 'NEGATIVE', 'score': 0.9859856963157654}]",0.985986,NEGATIVE,0,"[rt, carnecrudaradio, quiero, felicitar, al, alcalde, almeidapp, por, su, exitosa, labor, destru..."
1,2020-09-29 07:01:33,1310837099189473280,Señores de BiciMAD MADRID las bicis están muy descuidadas lo saben A menor uso de bicis más de m...,Neuroneater,Se ores de las bicis est n muy descuidadas lo saben A menor uso de bicis m s de metro y bus y,"[{'label': 'NEGATIVE', 'score': 0.9817882180213928}]",0.981788,NEGATIVE,0,"[señores, de, bicimad, madrid, las, bicis, están, muy, descuidadas, lo, saben, a, menor, uso, de..."
2,2020-09-29 07:43:50,1310847740386201600,JMDLatina Espero de este distrito no solo que proteja el mejor legado ambiental de Madrid sino q...,Andrés Pina,Espero de este distrito no solo que proteja el mejor legado ambiental de Madrid sino que incorpo...,"[{'label': 'NEGATIVE', 'score': 0.9809810519218445}]",0.980981,NEGATIVE,0,"[jmdlatina, espero, de, este, distrito, no, solo, que, proteja, el, mejor, legado, ambiental, de..."
3,2020-09-29 07:53:20,1310850131344920576,RT AguilarM PlataformaEMT BiciMAD bcarabante AlmeidaPP EMTmadrid O la fecha de la última OPE par...,ElMaNDaLoRiaNo,RT AguilarM O la fecha de la ltima OPE para Avituallamiento y Limpieza HACE,"[{'label': 'NEGATIVE', 'score': 0.9831892251968384}]",0.983189,NEGATIVE,1,"[rt, aguilarm, plataformaemt, bicimad, bcarabante, almeidapp, emtmadrid, o, la, fecha, de, la, ú..."
4,2020-09-29 08:05:56,1310853301810888704,La misma vergüenza de TODOS los días Una estación de BiciMAD totalmente abandonada por MADRID Al...,Diego Azul,La misma verg enza de TODOS los d as Una estaci n de totalmente abandonada por igua,"[{'label': 'NEGATIVE', 'score': 0.9943681955337524}]",0.994368,NEGATIVE,1,"[la, misma, vergüenza, de, todos, los, días, una, estación, de, bicimad, totalmente, abandonada,..."


In [17]:
import nltk

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Blanca/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
from nltk.corpus import stopwords

In [20]:
stopwords.fileids()

['arabic',
 'azerbaijani',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'greek',
 'hungarian',
 'indonesian',
 'italian',
 'kazakh',
 'nepali',
 'norwegian',
 'portuguese',
 'romanian',
 'russian',
 'slovene',
 'spanish',
 'swedish',
 'tajik',
 'turkish']

In [21]:
stopword = nltk.corpus.stopwords.words('spanish')

In [22]:
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    
df['Tweet_nonstop'] = df['Tweet_tokenized'].apply(lambda x: remove_stopwords(x))
df.head(10)

,date,id,text,user_name,tweets_clean,sentiment,score,label,ADR_label,Tweet_tokenized,Tweet_nonstop
0,2020-09-29 06:34:23,1310830261450539009,RT carnecrudaradio Quiero felicitar al alcalde AlmeidaPP por su exitosa labor destruyendo BiciMA...,alex vega,RT Quiero felicitar al alcalde por su exitosa labor destruyendo un servicio que funcionaba y que vu,"[{'label': 'NEGATIVE', 'score': 0.9859856963157654}]",0.985986,NEGATIVE,0,"[rt, carnecrudaradio, quiero, felicitar, al, alcalde, almeidapp, por, su, exitosa, labor, destru...","[rt, carnecrudaradio, quiero, felicitar, alcalde, almeidapp, exitosa, labor, destruyendo, bicima..."
1,2020-09-29 07:01:33,1310837099189473280,Señores de BiciMAD MADRID las bicis están muy descuidadas lo saben A menor uso de bicis más de m...,Neuroneater,Se ores de las bicis est n muy descuidadas lo saben A menor uso de bicis m s de metro y bus y,"[{'label': 'NEGATIVE', 'score': 0.9817882180213928}]",0.981788,NEGATIVE,0,"[señores, de, bicimad, madrid, las, bicis, están, muy, descuidadas, lo, saben, a, menor, uso, de...","[señores, bicimad, madrid, bicis, descuidadas, saben, menor, uso, bicis, metro, bus, httpstcokqd..."
2,2020-09-29 07:43:50,1310847740386201600,JMDLatina Espero de este distrito no solo que proteja el mejor legado ambiental de Madrid sino q...,Andrés Pina,Espero de este distrito no solo que proteja el mejor legado ambiental de Madrid sino que incorpo...,"[{'label': 'NEGATIVE', 'score': 0.9809810519218445}]",0.980981,NEGATIVE,0,"[jmdlatina, espero, de, este, distrito, no, solo, que, proteja, el, mejor, legado, ambiental, de...","[jmdlatina, espero, distrito, solo, proteja, mejor, legado, ambiental, madrid, sino, incorpore, ..."
3,2020-09-29 07:53:20,1310850131344920576,RT AguilarM PlataformaEMT BiciMAD bcarabante AlmeidaPP EMTmadrid O la fecha de la última OPE par...,ElMaNDaLoRiaNo,RT AguilarM O la fecha de la ltima OPE para Avituallamiento y Limpieza HACE,"[{'label': 'NEGATIVE', 'score': 0.9831892251968384}]",0.983189,NEGATIVE,1,"[rt, aguilarm, plataformaemt, bicimad, bcarabante, almeidapp, emtmadrid, o, la, fecha, de, la, ú...","[rt, aguilarm, plataformaemt, bicimad, bcarabante, almeidapp, emtmadrid, fecha, última, ope, avi..."
4,2020-09-29 08:05:56,1310853301810888704,La misma vergüenza de TODOS los días Una estación de BiciMAD totalmente abandonada por MADRID Al...,Diego Azul,La misma verg enza de TODOS los d as Una estaci n de totalmente abandonada por igua,"[{'label': 'NEGATIVE', 'score': 0.9943681955337524}]",0.994368,NEGATIVE,1,"[la, misma, vergüenza, de, todos, los, días, una, estación, de, bicimad, totalmente, abandonada,...","[misma, vergüenza, días, estación, bicimad, totalmente, abandonada, madrid, almeidapp, igua, htt..."
5,2020-09-29 08:13:49,1310855286345928706,RT carnecrudaradio Quiero felicitar al alcalde AlmeidaPP por su exitosa labor destruyendo BiciMA...,Juan talavera,RT Quiero felicitar al alcalde por su exitosa labor destruyendo un servicio que funcionaba y que vu,"[{'label': 'NEGATIVE', 'score': 0.9859856963157654}]",0.985986,NEGATIVE,1,"[rt, carnecrudaradio, quiero, felicitar, al, alcalde, almeidapp, por, su, exitosa, labor, destru...","[rt, carnecrudaradio, quiero, felicitar, alcalde, almeidapp, exitosa, labor, destruyendo, bicima..."
6,2020-09-29 08:13:52,1310855298777837568,RT carnecrudaradio Quiero felicitar al alcalde AlmeidaPP por su exitosa labor destruyendo BiciMA...,Susie Vicha,RT Quiero felicitar al alcalde por su exitosa labor destruyendo un servicio que funcionaba y que vu,"[{'label': 'NEGATIVE', 'score': 0.9859856963157654}]",0.985986,NEGATIVE,0,"[rt, carnecrudaradio, quiero, felicitar, al, alcalde, almeidapp, por, su, exitosa, labor, destru...","[rt, carnecrudaradio, quiero, felicitar, alcalde, almeidapp, exitosa, labor, destruyendo, bicima..."
7,2020-09-29 08:50:57,1310864629661343744,RT diegorebollo La misma vergüenza de TODOS los días Una estación de BiciMAD totalmente abandona...,Carlos Martin 🔻,RT rebollo La misma verg enza de TODOS los d as Una esta

In [23]:
ps = nltk.PorterStemmer()
def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

df['Tweet_stemmed'] = df['Tweet_nonstop'].apply(lambda x: stemming(x))
df.head()

,date,id,text,user_name,tweets_clean,sentiment,score,label,ADR_label,Tweet_tokenized,Tweet_nonstop,Tweet_stemmed
0,2020-09-29 06:34:23,1310830261450539009,RT carnecrudaradio Quiero felicitar al alcalde AlmeidaPP por su exitosa labor destruyendo BiciMA...,alex vega,RT Quiero felicitar al alcalde por su exitosa labor destruyendo un servicio que funcionaba y que vu,"[{'label': 'NEGATIVE', 'score': 0.9859856963157654}]",0.985986,NEGATIVE,0,"[rt, carnecrudaradio, quiero, felicitar, al, alcalde, almeidapp, por, su, exitosa, labor, destru...","[rt, carnecrudaradio, quiero, felicitar, alcalde, almeidapp, exitosa, labor, destruyendo, bicima...","[rt, carnecrudaradio, quiero, felicitar, alcald, almeidapp, exitosa, labor, destruyendo, bicimad..."
1,2020-09-29 07:01:33,1310837099189473280,Señores de BiciMAD MADRID las bicis están muy descuidadas lo saben A menor uso de bicis más de m...,Neuroneater,Se ores de las bicis est n muy descuidadas lo saben A menor uso de bicis m s de metro y bus y,"[{'label': 'NEGATIVE', 'score': 0.9817882180213928}]",0.981788,NEGATIVE,0,"[señores, de, bicimad, madrid, las, bicis, están, muy, descuidadas, lo, saben, a, menor, uso, de...","[señores, bicimad, madrid, bicis, descuidadas, saben, menor, uso, bicis, metro, bus, httpstcokqd...","[señor, bicimad, madrid, bici, descuidada, saben, menor, uso, bici, metro, bu, httpstcokqddneij]"
2,2020-09-29 07:43:50,1310847740386201600,JMDLatina Espero de este distrito no solo que proteja el mejor legado ambiental de Madrid sino q...,Andrés Pina,Espero de este distrito no solo que proteja el mejor legado ambiental de Madrid sino que incorpo...,"[{'label': 'NEGATIVE', 'score': 0.9809810519218445}]",0.980981,NEGATIVE,0,"[jmdlatina, espero, de, este, distrito, no, solo, que, proteja, el, mejor, legado, ambiental, de...","[jmdlatina, espero, distrito, solo, proteja, mejor, legado, ambiental, madrid, sino, incorpore, ...","[jmdlatina, espero, distrito, solo, proteja, mejor, legado, ambient, madrid, sino, incorpor, htt..."
3,2020-09-29 07:53:20,1310850131344920576,RT AguilarM PlataformaEMT BiciMAD bcarabante AlmeidaPP EMTmadrid O la fecha de la última OPE par...,ElMaNDaLoRiaNo,RT AguilarM O la fecha de la ltima OPE para Avituallamiento y Limpieza HACE,"[{'label': 'NEGATIVE', 'score': 0.9831892251968384}]",0.983189,NEGATIVE,1,"[rt, aguilarm, plataformaemt, bicimad, bcarabante, almeidapp, emtmadrid, o, la, fecha, de, la, ú...","[rt, aguilarm, plataformaemt, bicimad, bcarabante, almeidapp, emtmadrid, fecha, última, ope, avi...","[rt, aguilarm, plataformaemt, bicimad, bcarabant, almeidapp, emtmadrid, fecha, última, ope, avit..."
4,2020-09-29 08:05:56,1310853301810888704,La misma vergüenza de TODOS los días Una estación de BiciMAD totalmente abandonada por MADRID Al...,Diego Azul,La misma verg enza de TODOS los d as Una estaci n de totalmente abandonada por igua,"[{'label': 'NEGATIVE', 'score': 0.9943681955337524}]",0.994368,NEGATIVE,1,"[la, misma, vergüenza, de, todos, los, días, una, estación, de, bicimad, totalmente, abandonada,...","[misma, vergüenza, días, estación, bicimad, totalmente, abandonada, madrid, almeidapp, igua, htt...","[misma, vergüenza, día, estación, bicimad, totalment, abandonada, madrid, almeidapp, igua, https..."


In [24]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/Blanca/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [25]:
wn = nltk.WordNetLemmatizer()
def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text

df['Tweet_lemmatized'] = df['Tweet_nonstop'].apply(lambda x: lemmatizer(x))
df.head()

,date,id,text,user_name,tweets_clean,sentiment,score,label,ADR_label,Tweet_tokenized,Tweet_nonstop,Tweet_stemmed,Tweet_lemmatized
0,2020-09-29 06:34:23,1310830261450539009,RT carnecrudaradio Quiero felicitar al alcalde AlmeidaPP por su exitosa labor destruyendo BiciMA...,alex vega,RT Quiero felicitar al alcalde por su exitosa labor destruyendo un servicio que funcionaba y que vu,"[{'label': 'NEGATIVE', 'score': 0.9859856963157654}]",0.985986,NEGATIVE,0,"[rt, carnecrudaradio, quiero, felicitar, al, alcalde, almeidapp, por, su, exitosa, labor, destru...","[rt, carnecrudaradio, quiero, felicitar, alcalde, almeidapp, exitosa, labor, destruyendo, bicima...","[rt, carnecrudaradio, quiero, felicitar, alcald, almeidapp, exitosa, labor, destruyendo, bicimad...","[rt, carnecrudaradio, quiero, felicitar, alcalde, almeidapp, exitosa, labor, destruyendo, bicima..."
1,2020-09-29 07:01:33,1310837099189473280,Señores de BiciMAD MADRID las bicis están muy descuidadas lo saben A menor uso de bicis más de m...,Neuroneater,Se ores de las bicis est n muy descuidadas lo saben A menor uso de bicis m s de metro y bus y,"[{'label': 'NEGATIVE', 'score': 0.9817882180213928}]",0.981788,NEGATIVE,0,"[señores, de, bicimad, madrid, las, bicis, están, muy, descuidadas, lo, saben, a, menor, uso, de...","[señores, bicimad, madrid, bicis, descuidadas, saben, menor, uso, bicis, metro, bus, httpstcokqd...","[señor, bicimad, madrid, bici, descuidada, saben, menor, uso, bici, metro, bu, httpstcokqddneij]","[señores, bicimad, madrid, bicis, descuidadas, saben, menor, uso, bicis, metro, bus, httpstcokqd..."
2,2020-09-29 07:43:50,1310847740386201600,JMDLatina Espero de este distrito no solo que proteja el mejor legado ambiental de Madrid sino q...,Andrés Pina,Espero de este distrito no solo que proteja el mejor legado ambiental de Madrid sino que incorpo...,"[{'label': 'NEGATIVE', 'score': 0.9809810519218445}]",0.980981,NEGATIVE,0,"[jmdlatina, espero, de, este, distrito, no, solo, que, proteja, el, mejor, legado, ambiental, de...","[jmdlatina, espero, distrito, solo, proteja, mejor, legado, ambiental, madrid, sino, incorpore, ...","[jmdlatina, espero, distrito, solo, proteja, mejor, legado, ambient, madrid, sino, incorpor, htt...","[jmdlatina, espero, distrito, solo, proteja, mejor, legado, ambiental, madrid, sino, incorpore, ..."
3,2020-09-29 07:53:20,1310850131344920576,RT AguilarM PlataformaEMT BiciMAD bcarabante AlmeidaPP EMTmadrid O la fecha de la última OPE par...,ElMaNDaLoRiaNo,RT AguilarM O la fecha de la ltima OPE para Avituallamiento y Limpieza HACE,"[{'label': 'NEGATIVE', 'score': 0.9831892251968384}]",0.983189,NEGATIVE,1,"[rt, aguilarm, plataformaemt, bicimad, bcarabante, almeidapp, emtmadrid, o, la, fecha, de, la, ú...","[rt, aguilarm, plataformaemt, bicimad, bcarabante, almeidapp, emtmadrid, fecha, última, ope, avi...","[rt, aguilarm, plataformaemt, bicimad, bcarabant, almeidapp, emtmadrid, fecha, última, ope, avit...","[rt, aguilarm, plataformaemt, bicimad, bcarabante, almeidapp, emtmadrid, fecha, última, ope, avi..."
4,2020-09-29 08:05:56,1310853301810888704,La misma vergüenza de TODOS los días Una estación de BiciMAD totalmente abandonada por MADRID Al...,Diego Azul,La misma verg enza de TODOS los d as Una estaci n de totalmente abandonada por igua,"[{'label': 'NEGATIVE', 'score': 0.9943681955337524}]",0.994368,NEGATIVE,1,"[la, misma, vergüenza, de, todos, los, días, una, estación, de, bicimad, totalmente, abandonada,...","[misma, vergüenza, días, estación, bicimad, totalmente, abandonada, madrid, almeidapp, igua, htt...","[misma, vergüenza, día, estación, bicimad, totalment, abandonada, madrid, almeidapp, igua, https...","[misma, vergüenza, días, estación, bicimad, totalmente, abandonada, madrid, almeidapp, igua, htt..."


In [26]:
def clean_text(text):
    text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
    text_rc = re.sub('[0-9]+', '', text_lc)
    tokens = re.split('\W+', text_rc)    # tokenization
    text = [ps.stem(word) for word in tokens if word not in stopword]  # remove stopwords and stemming
    return text

In [27]:
countVectorizer = CountVectorizer(analyzer=clean_text) 
countVector = countVectorizer.fit_transform(df['text'])
print('{} Number of tweets has {} words'.format(countVector.shape[0], countVector.shape[1]))

6949 Number of tweets has 5169 words


In [ ]:
# https://www.kaggle.com/ragnisah/text-data-cleaning-tweets-analysis

In [28]:
count_vect_df = pd.DataFrame(countVector.toarray(), columns=countVectorizer.get_feature_names())
count_vect_df.head()

,,aavv,ab,ababol,abajo,abandona,abandonada,abandonado,abandonar,abandono,...,éxito,ínfima,íñigo,última,últimam,últimament,último,únans,única,único
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
df = pd.DataFrame(df[['id', 'Tweet_lemmatized']])

In [40]:
df["Tweet_lemmatized"] = df.Tweet_lemmatized.to_string()

In [ ]:
from wordcloud import WordCloud, STOPWORDS , ImageColorGenerator
# Start with one review:
df_ADR = df[df['ADR_label']==1]
df_NADR = df[df['ADR_label']==0]
tweet_All = " ".join(review for review in df.Tweet_lemmatized)
tweet_ADR = " ".join(review for review in df_ADR.Tweet_lemmatized)
tweet_NADR = " ".join(review for review in df_NADR.Tweet_lemmatized)

fig, ax = plt.subplots(3, 1, figsize  = (30,30))
# Create and generate a word cloud image:
wordcloud_ALL = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(tweet_All)
wordcloud_ADR = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(tweet_ADR)
wordcloud_NADR = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(tweet_NADR)

# Display the generated image:
ax[0].imshow(wordcloud_ALL, interpolation='bilinear')
ax[0].set_title('All Tweets', fontsize=30)
ax[0].axis('off')
ax[1].imshow(wordcloud_ADR, interpolation='bilinear')
ax[1].set_title('Tweets under ADR Class',fontsize=30)
ax[1].axis('off')
ax[2].imshow(wordcloud_NADR, interpolation='bilinear')
ax[2].set_title('Tweets under None - ADR Class',fontsize=30)
ax[2].axis('off')

wordcloud.to_file("../results/first_review.png")

In [ ]:
sns.countplot(x = 'ADR_label', data = tweet_df)

In [ ]:
df = pd.DataFrame(tweet_df[['id', 'text']])

In [ ]:
from wordcloud import WordCloud, STOPWORDS , ImageColorGenerator
# Start with one review:
df_ADR = tweet_df[tweet_df['ADR_label']==1]
df_NADR = tweet_df[tweet_df['ADR_label']==0]
tweet_All = " ".join(review for review in df.Tweet)
tweet_ADR = " ".join(review for review in df_ADR.Tweet)
tweet_NADR = " ".join(review for review in df_NADR.Tweet)

fig, ax = plt.subplots(3, 1, figsize  = (30,30))
# Create and generate a word cloud image:
wordcloud_ALL = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(tweet_All)
wordcloud_ADR = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(tweet_ADR)
wordcloud_NADR = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(tweet_NADR)

# Display the generated image:
ax[0].imshow(wordcloud_ALL, interpolation='bilinear')
ax[0].set_title('All Tweets', fontsize=30)
ax[0].axis('off')
ax[1].imshow(wordcloud_ADR, interpolation='bilinear')
ax[1].set_title('Tweets under ADR Class',fontsize=30)
ax[1].axis('off')
ax[2].imshow(wordcloud_NADR, interpolation='bilinear')
ax[2].set_title('Tweets under None - ADR Class',fontsize=30)
ax[2].axis('off')

#wordcloud.to_file("img/first_review.png")

In [36]:
# Start with one review:
text = df.tex[0]

# Create and generate a word cloud image:
wordcloud = WordCloud().generate(text)

# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

NameError: name 'text' is not defined